# Machine Learning on Spark – MLLib

As organizations create more diverse and more user-focused data products and services, there is a growing need for machine learning, which can be used to develop personalizations, recommendations, and predictive insights. The Apache Spark machine learning library (MLlib) allows data scientists to focus on their data problems and models instead of solving the complexities surrounding distributed data (such as infrastructure, configurations, and so on).

From the inception of the Apache Spark project, MLlib was considered foundational for Spark’s success. The key benefit of MLlib is that it allows data scientists to focus on their data problems and models instead of solving the complexities surrounding distributed data (such as infrastructure, configurations, and so on). The data engineers can focus on distributed systems engineering using Spark’s easy-to-use APIs, while the data scientists can leverage the scale and speed of Spark core. Just as important, Spark MLlib is a general-purpose library, providing algorithms for most use cases while at the same time allowing the community to build upon and extend it for specialized use cases.

This week we will look at the basics of some of the spark MLlib packages, next week we build these into machine learning pipelines.

We will start with a databricks database that compares city population to median sale prices of homes.

In [0]:
import pyspark
from pyspark.sql.functions import col
# Use the Spark CSV datasource with options specifying:
# - First line of file is a header
# - Automatically infer the schema of the data
data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/databricks-datasets/samples/population-vs-price/data_geo.csv")
data.cache() # Cache data for faster reuse
data.count()

Out[1]: 294

Take a look at the data schema.

In [0]:
data.printSchema()
data.take(10)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

Out[2]: [Row(2014 rank=101, City='Birmingham', State='Alabama', State Code='AL', 2014 Population estimate=212247, 2015 median sales price=162.9),
 Row(2014 rank=125, City='Huntsville', State='Alabama', State Code='AL', 2014 Population estimate=188226, 2015 median sales price=157.7),
 Row(2014 rank=122, City='Mobile', State='Alabama', State Code='AL', 2014 Population estimate=194675, 2015 median sales price=122.5),
 Row(2014 rank=114, City='Montgomery', State='Alabama', State Code='AL', 2014 Population estimate=200481, 2015 median sales price=129.0),
 Row(2014 rank=64, City='Anchorage[19]', State='Alaska', State Code='AK', 2014 Population estimate=301010, 2015 median sales price=None),
 Row(2014 rank=78, City='Chandl

Here we clean the data by dropping any rows with missing values.

In [0]:
data = data.dropna() # drop rows with missing values
data.count()

Out[3]: 109

Next we select the columns we will use to predict house prices (features) and the correct output (labels). Below we use the databricks display function which can be useful for examining our dataframes.

In [0]:
dataFeat = data.select(["2014 Population estimate","2015 median sales price"])
display(dataFeat)

2014 Population estimate,2015 median sales price
212247,162.9
188226,157.7
194675,122.5
200481,129.0
1537058,206.1
527972,178.1
197706,131.8
346997,685.7
3928864,434.7
319504,281.0


Finally we need to convert the data to the format expected by spark, a vector of features as a single column and the label as an additional column.

We define a function that does this (which we can use later). As you can see it takes our DataFrame and converts it to an RDD. This then allows us to apply the map transformation to each entry of the RDD. Here we take all the columns except the last (in our case only one) and convert these to a dense vector, we keep the last column seperate for the label.

Vectors (dense or sparse) are a way of holding data that is used a lot in machine learning. In the features column below we only have one data item (the size of the population) but as you can see the data structure is as follows: [1,1,[],[188226]] this format makes more sense when we use sparse vectors (it easier to see the data is one of the advantages, but mostly its is more space efficient) for more information this video has a short explanation of the vector format: https://www.youtube.com/watch?v=oGwEv82ifrE

Here is the same vector in three formats:
```python
# Use a NumPy array as a dense vector.
dv1 = np.array([1.0, 0.0, 3.0])
# Use a Python list as a dense vector.
dv2 = [1.0, 0.0, 3.0]
# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0]) 
```
You will find out that most of the machine learning algorithms in Spark are based on the features and label format. That is to say, you can play with all of the machine learning algorithms in Spark aftre you prepare your data in this format with features and label.

The function defined below will take a row of our dataframe and convert all of the features into a dense vector. The final dataformat is a dataframe with two columns, a label column and a features column.

https://spark.apache.org/docs/latest/ml-guide.html

In [0]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [0]:
a = dataFeat.rdd.map(lambda x: Vectors.dense(x))
a.collect()

Out[6]: [DenseVector([212247.0, 162.9]),
 DenseVector([188226.0, 157.7]),
 DenseVector([194675.0, 122.5]),
 DenseVector([200481.0, 129.0]),
 DenseVector([1537058.0, 206.1]),
 DenseVector([527972.0, 178.1]),
 DenseVector([197706.0, 131.8]),
 DenseVector([346997.0, 685.7]),
 DenseVector([3928864.0, 434.7]),
 DenseVector([319504.0, 281.0]),
 DenseVector([485199.0, 275.8]),
 DenseVector([1381069.0, 510.3]),
 DenseVector([852469.0, 748.3]),
 DenseVector([1015785.0, 900.0]),
 DenseVector([105112.0, 442.2]),
 DenseVector([445830.0, 220.1]),
 DenseVector([663862.0, 338.1]),
 DenseVector([147612.0, 342.7]),
 DenseVector([124705.0, 202.6]),
 DenseVector([130282.0, 205.1]),
 DenseVector([658893.0, 367.8]),
 DenseVector([169854.0, 208.0]),
 DenseVector([128460.0, 169.9]),
 DenseVector([853382.0, 182.5]),
 DenseVector([102346.0, 136.0]),
 DenseVector([430332.0, 269.1]),
 DenseVector([105838.0, 145.0]),
 DenseVector([174110.0, 172.0]),
 DenseVector([188107.0, 170.5]),
 DenseVector([358699.0, 156.0])

In [0]:
def transData(data):
  # Combine columns to a dense vector (excluding the last column)
  dataFeaturesRDD = data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]])
  
  # Convert the RDD back to a DataFrame, labelling the columns
  featuresDF =  dataFeaturesRDD.toDF(['features','label'])
  
  return featuresDF

In [0]:
dataLR = transData(dataFeat)
dataLR.show()

+-----------+-----+
|   features|label|
+-----------+-----+
| [212247.0]|162.9|
| [188226.0]|157.7|
| [194675.0]|122.5|
| [200481.0]|129.0|
|[1537058.0]|206.1|
| [527972.0]|178.1|
| [197706.0]|131.8|
| [346997.0]|685.7|
|[3928864.0]|434.7|
| [319504.0]|281.0|
| [485199.0]|275.8|
|[1381069.0]|510.3|
| [852469.0]|748.3|
|[1015785.0]|900.0|
| [105112.0]|442.2|
| [445830.0]|220.1|
| [663862.0]|338.1|
| [147612.0]|342.7|
| [124705.0]|202.6|
| [130282.0]|205.1|
+-----------+-----+
only showing top 20 rows



Finally we can run our linear regression. Here we create two models with different parameters and print the coefficients they provide.

In [0]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

# Fit 2 models, using different regularization parameters
modelA = lr.fit(dataLR, {lr.regParam:0.0})
modelB = lr.fit(dataLR, {lr.regParam:100.0})

# Print the fitted model parameters
print(">>>> ModelA intercept: %r, coefficient: %r" % (modelA.intercept, modelA.coefficients[0]))
print(">>>> ModelB intercept: %r, coefficient: %r" % (modelB.intercept, modelB.coefficients[0]))

>>>> ModelA intercept: 191.29427575139394, coefficient: 3.779789682338248e-05
>>>> ModelB intercept: 199.85112564667153, coefficient: 2.160349948371716e-05


Sometimes we will want to see the significance of the coefficients we have. We can check the p values of our regressors:

In [0]:
seA = modelA.summary.pValues
seB = modelB.summary.pValues

print(">>>> ModelA p value intercept: %r, p value coefficient: %r" % (seA[1], seA[0]))
print(">>>> ModelB p value intercept: %r, p value coefficient: %r" % (seB[1], seB[0]))

>>>> ModelA p value intercept: 0.0, p value coefficient: 0.005212735655699907
>>>> ModelB p value intercept: 0.0, p value coefficient: 0.0345130599830199


Generating predictions based on the model is easy, here we use the input data and compare model predicted to actual values.

In [0]:
predictionsA = modelA.transform(dataLR)
predictionsB = modelB.transform(dataLR)

In [0]:
display(predictionsA)

features,label,prediction
"Map(vectorType -> dense, length -> 1, values -> List(212247.0))",162.9,199.3167659584664
"Map(vectorType -> dense, length -> 1, values -> List(188226.0))",157.7,198.40882267887193
"Map(vectorType -> dense, length -> 1, values -> List(194675.0))",122.5,198.65258131548592
"Map(vectorType -> dense, length -> 1, values -> List(200481.0))",129.0,198.87203590444247
"Map(vectorType -> dense, length -> 1, values -> List(1537058.0))",206.1,249.39183544694856
"Map(vectorType -> dense, length -> 1, values -> List(527972.0))",178.1,211.25050693302884
"Map(vectorType -> dense, length -> 1, values -> List(197706.0))",131.8,198.7671467407576
"Map(vectorType -> dense, length -> 1, values -> List(346997.0))",685.7,204.4100325554172
"Map(vectorType -> dense, length -> 1, values -> List(3928864.0))",434.7,339.79707185649573
"Map(vectorType -> dense, length -> 1, values -> List(319504.0))",281.0,203.37085497805194


In [0]:
display(predictionsB)

features,label,prediction
"Map(vectorType -> dense, length -> 1, values -> List(212247.0))",162.9,204.43640360159205
"Map(vectorType -> dense, length -> 1, values -> List(188226.0))",157.7,203.91746594049368
"Map(vectorType -> dense, length -> 1, values -> List(194675.0))",122.5,204.05678690866418
"Map(vectorType -> dense, length -> 1, values -> List(200481.0))",129.0,204.18221682666663
"Map(vectorType -> dense, length -> 1, values -> List(1537058.0))",206.1,233.05695735611488
"Map(vectorType -> dense, length -> 1, values -> List(527972.0))",178.1,211.25716847608865
"Map(vectorType -> dense, length -> 1, values -> List(197706.0))",131.8,204.12226711559933
"Map(vectorType -> dense, length -> 1, values -> List(346997.0))",685.7,207.34747515702293
"Map(vectorType -> dense, length -> 1, values -> List(3928864.0))",434.7,284.7283370422665
"Map(vectorType -> dense, length -> 1, values -> List(319504.0))",281.0,206.7535301457171


The predictions dont look too great, in databricks we can easily plot the residuals. You can drag the chart to make it bigger.

In [0]:
display(modelB, dataLR)
#they should have a random distribution around 0

fitted values,residuals
204.43640360159205,-41.53640360159204
203.91746594049368,-46.217465940493696
204.05678690866418,-81.55678690866418
204.18221682666663,-75.18221682666663
233.05695735611488,-26.956957356114884
211.25716847608865,-33.15716847608866
204.12226711559933,-72.32226711559932
207.34747515702293,478.3525248429771
284.7283370422665,149.97166295773349
206.7535301457171,74.2464698542829


Additionaly, Spark includes functionality to help us with our models, for example the regression evaluator.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse")
RMSE_A = evaluator.evaluate(predictionsA)
RMSE_B = evaluator.evaluate(predictionsB)

print("ModelA: Root Mean Squared Error = " + str(RMSE_A))
print("ModelB: Root Mean Squared Error = " + str(RMSE_B))

#Root Mean Square Error (RMSE) is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. 

ModelA: Root Mean Squared Error = 128.60202684284758
ModelB: Root Mean Squared Error = 129.49630019270606


According to RMSE ModelA performs better.


### Practice exercise

In the below two cells we download a database of car performance (efficiency in miles per gallon MPG) and a number of car characteristics. Use the linear regression estimator to try to predict miles per gallon MPG based on:

 - Just <b> weight </b> as a feature

 - <b> displacement, weight, acceleration, and year </b> as features

Compare the RMSE of your the two aproaches and plot the residuals.

Hint: the `transData` function we defined is very useful, however, it requires the final column to be the data we are trying to predict. You will need to either change this function or re-arrange the dataframe, as current, MPG is the first column.

In [0]:
%sh wget https://raw.githubusercontent.com/plotly/datasets/master/auto-mpg.csv

--2024-05-18 19:30:02--  https://raw.githubusercontent.com/plotly/datasets/master/auto-mpg.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10118 (9.9K) [text/plain]
Saving to: ‘auto-mpg.csv’

     0K .........                                             100% 37.2M=0s

2024-05-18 19:30:02 (37.2 MB/s) - ‘auto-mpg.csv’ saved [10118/10118]



In [0]:
dataMPG = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("file:/databricks/driver/auto-mpg.csv")
dataMPG.cache()
print(dataMPG.count())
dataMPG.printSchema()

398
root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model-year: integer (nullable = true)



In [0]:
display(dataMPG.select([dataMPG.columns[2], dataMPG.columns[4], dataMPG.columns[5],  dataMPG.columns[6], dataMPG.columns[0]]))

displacement,weight,acceleration,model-year,mpg
307.0,3504,12.0,70,18.0
350.0,3693,11.5,70,15.0
318.0,3436,11.0,70,18.0
304.0,3433,12.0,70,16.0
302.0,3449,10.5,70,17.0
429.0,4341,10.0,70,15.0
454.0,4354,9.0,70,14.0
440.0,4312,8.5,70,14.0
455.0,4425,10.0,70,14.0
390.0,3850,8.5,70,15.0


In [0]:
data1 = transData(dataMPG.select([dataMPG.columns[4], dataMPG.columns[0]]))
display(data1)

features,label
"Map(vectorType -> dense, length -> 1, values -> List(3504.0))",18.0
"Map(vectorType -> dense, length -> 1, values -> List(3693.0))",15.0
"Map(vectorType -> dense, length -> 1, values -> List(3436.0))",18.0
"Map(vectorType -> dense, length -> 1, values -> List(3433.0))",16.0
"Map(vectorType -> dense, length -> 1, values -> List(3449.0))",17.0
"Map(vectorType -> dense, length -> 1, values -> List(4341.0))",15.0
"Map(vectorType -> dense, length -> 1, values -> List(4354.0))",14.0
"Map(vectorType -> dense, length -> 1, values -> List(4312.0))",14.0
"Map(vectorType -> dense, length -> 1, values -> List(4425.0))",14.0
"Map(vectorType -> dense, length -> 1, values -> List(3850.0))",15.0


In [0]:
data2 = transData(dataMPG.select([dataMPG.columns[2], dataMPG.columns[4], dataMPG.columns[5],  dataMPG.columns[6], dataMPG.columns[0]]))
display(data2)

features,label
"Map(vectorType -> dense, length -> 4, values -> List(307.0, 3504.0, 12.0, 70.0))",18.0
"Map(vectorType -> dense, length -> 4, values -> List(350.0, 3693.0, 11.5, 70.0))",15.0
"Map(vectorType -> dense, length -> 4, values -> List(318.0, 3436.0, 11.0, 70.0))",18.0
"Map(vectorType -> dense, length -> 4, values -> List(304.0, 3433.0, 12.0, 70.0))",16.0
"Map(vectorType -> dense, length -> 4, values -> List(302.0, 3449.0, 10.5, 70.0))",17.0
"Map(vectorType -> dense, length -> 4, values -> List(429.0, 4341.0, 10.0, 70.0))",15.0
"Map(vectorType -> dense, length -> 4, values -> List(454.0, 4354.0, 9.0, 70.0))",14.0
"Map(vectorType -> dense, length -> 4, values -> List(440.0, 4312.0, 8.5, 70.0))",14.0
"Map(vectorType -> dense, length -> 4, values -> List(455.0, 4425.0, 10.0, 70.0))",14.0
"Map(vectorType -> dense, length -> 4, values -> List(390.0, 3850.0, 8.5, 70.0))",15.0


In [0]:
modelM1 = lr.fit(data1, {lr.regParam:0.0})
modelM2 = lr.fit(data2, {lr.regParam:0.0})

predictionsM1 = modelM1.transform(data1)
predictionsM2 = modelM2.transform(data2)

In [0]:
print(">>>> Model intercept: %r, coefficient: %r" % (modelM1.intercept, modelM1.coefficients))
print(">>>> Model intercept: %r, coefficient: %r" % (modelM2.intercept, modelM2.coefficients))

>>>> Model intercept: 46.31736442026568, coefficient: DenseVector([-0.0077])
>>>> Model intercept: -15.130597291474587, coefficient: DenseVector([0.0035, -0.007, 0.0791, 0.7553])


In [0]:
RMSE_M1 = evaluator.evaluate(predictionsM1)
RMSE_M2 = evaluator.evaluate(predictionsM2)

print("Model1: Root Mean Squared Error = " + str(RMSE_M1))
print("Model2: Root Mean Squared Error = " + str(RMSE_M2))

Model1: Root Mean Squared Error = 4.3336981591509565
Model2: Root Mean Squared Error = 3.417052956459645


### Gradient Boosted Tree Regression

Lets try another model from the spark toolkit on our data: Gradient Boosted Tree Regression https://statisticasoftware.wordpress.com/2012/09/11/boosting-trees-for-regression-and-classification/

Follow the steps below to see how easily we can apply a different machine learning approach to our data.

In [0]:
# Import LinearRegression class
from pyspark.ml.regression import GBTRegressor

# Define LinearRegression algorithm
rf = GBTRegressor(maxDepth=2, seed=42) #1st try: maxDepth=2, seed=42 | 2nd try: maxDepth=2, seed=42 
dataMPGOrdered = transData(dataMPG.select(dataMPG.columns[1:] + [dataMPG.columns[0]]  ))
rf.setMaxIter(10) #1st try: 10 | 2nd try: 30
rf.setMinWeightFractionPerNode(0.1)  #1st try: 0.1 | 2nd try: 0.005
modelG = rf.fit(dataMPGOrdered)

In [0]:
predictions = modelG.transform(dataMPGOrdered)
predictions.select("features","label", "prediction").show(5)

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|[8.0,307.0,130.0,...| 18.0|13.850634402202939|
|[8.0,350.0,165.0,...| 15.0|13.850634402202939|
|[8.0,318.0,150.0,...| 18.0|13.850634402202939|
|[8.0,304.0,150.0,...| 16.0|13.850634402202939|
|[8.0,302.0,140.0,...| 17.0|13.850634402202939|
+--------------------+-----+------------------+
only showing top 5 rows



In [0]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

#1st 3.12151  | 2nd 0.847873

Root Mean Squared Error (RMSE) on test data = 3.15872


How did this approach perform in comparison to the previous regressions? Remember we are only looking at RMSE on the training data.

This model is prone for overfitting, below we split the data into a training and test dataset. This would be standard good practice if this is all of the data we have.

Fit the model to the training dataset and test the fit against the test dataset.

In [0]:
(trainingData, testData) = dataMPGOrdered.randomSplit([0.6, 0.4])

trainingData.show(5)
testData.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[3.0,70.0,90.0,21...| 18.0|
|[3.0,70.0,97.0,23...| 19.0|
|[3.0,70.0,100.0,2...| 23.7|
|[3.0,80.0,110.0,2...| 21.5|
|[4.0,68.0,49.0,18...| 29.0|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[4.0,72.0,69.0,16...| 35.0|
|[4.0,78.0,52.0,19...| 32.8|
|[4.0,79.0,67.0,19...| 26.0|
|[4.0,83.0,61.0,20...| 32.0|
|[4.0,85.0,52.0,20...| 29.0|
+--------------------+-----+
only showing top 5 rows



In [0]:
modelGTr = rf.fit(trainingData)
predictionsGTr = modelGTr.transform(testData)
rmseGtr = evaluator.evaluate(predictionsGTr)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGtr)

Root Mean Squared Error (RMSE) on test data = 3.92357


## Bonus exercise: Machine learning practice

I've included below a dataset of wine characteristics and quality, you can use this to practice applying the aprroachs above to build a predictor of wine quality.

Try to find the best model you can to predict wine quality.

If you want, you can try other machine learning models, see: https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#regression

##### Below I fit two basic models to the data, based on the methods we have seen in this lab, and compare them. However, there is a lot more you could do.

In [0]:
%sh wget https://raw.githubusercontent.com/MingChen0919/learning-apache-spark/master/data/WineData.csv

--2024-05-18 19:30:34--  https://raw.githubusercontent.com/MingChen0919/learning-apache-spark/master/data/WineData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84143 (82K) [text/plain]
Saving to: ‘WineData.csv’

     0K .......... .......... .......... .......... .......... 60% 6.39M 0s
    50K .......... .......... .......... ..                   100% 5.59M=0.01s

2024-05-18 19:30:34 (6.05 MB/s) - ‘WineData.csv’ saved [84143/84143]



In [0]:
wineDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("file:/databricks/driver/WineData.csv")
wineDF.cache()
wineDF.printSchema()
wineDF.select("quality").distinct().take(100)

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)

Out[30]: [Row(quality=6),
 Row(quality=3),
 Row(quality=5),
 Row(quality=4),
 Row(quality=8),
 Row(quality=7)]

In [0]:
# Convert to float format
def string_to_float(x):
    return float(x)

#
def condition(r):
    if (0<= r <= 4):
        label = "low"
    elif(4< r <= 6):
        label = "medium"
    else:
        label = "high"
    return label
  
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType
string_to_float_udf = udf(string_to_float, DoubleType())
quality_udf = udf(lambda x: condition(x), StringType())
wineDFQ = wineDF.withColumn("quality", quality_udf("quality"))

In [0]:
wineDF.show(10,True)
wineDF.printSchema()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [0]:
dataWine = transData(wineDF)
display(dataWine)

features,label
"Map(vectorType -> dense, length -> 11, values -> List(7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4))",5
"Map(vectorType -> dense, length -> 11, values -> List(7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.9968, 3.2, 0.68, 9.8))",5
"Map(vectorType -> dense, length -> 11, values -> List(7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.997, 3.26, 0.65, 9.8))",5
"Map(vectorType -> dense, length -> 11, values -> List(11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.998, 3.16, 0.58, 9.8))",6
"Map(vectorType -> dense, length -> 11, values -> List(7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4))",5
"Map(vectorType -> dense, length -> 11, values -> List(7.4, 0.66, 0.0, 1.8, 0.075, 13.0, 40.0, 0.9978, 3.51, 0.56, 9.4))",5
"Map(vectorType -> dense, length -> 11, values -> List(7.9, 0.6, 0.06, 1.6, 0.069, 15.0, 59.0, 0.9964, 3.3, 0.46, 9.4))",5
"Map(vectorType -> dense, length -> 11, values -> List(7.3, 0.65, 0.0, 1.2, 0.065, 15.0, 21.0, 0.9946, 3.39, 0.47, 10.0))",7
"Map(vectorType -> dense, length -> 11, values -> List(7.8, 0.58, 0.02, 2.0, 0.073, 9.0, 18.0, 0.9968, 3.36, 0.57, 9.5))",7
"Map(vectorType -> dense, length -> 11, values -> List(7.5, 0.5, 0.36, 6.1, 0.071, 17.0, 102.0, 0.9978, 3.35, 0.8, 10.5))",5


In [0]:
(trainingDataWine, testDataWine) = dataWine.randomSplit([0.6, 0.4])

In [0]:
# Gradient Boosted Trees
modelGBTWine = rf.fit(trainingDataWine)

In [0]:
predictionsGBTWine = modelGBTWine.transform(testDataWine)

In [0]:
rmseGBTWine = evaluator.evaluate(predictionsGBTWine)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGBTWine)

Root Mean Squared Error (RMSE) on test data = 0.663375


In [0]:
# Fit 2 models, using different regularization parameters
modelWineA = lr.fit(trainingDataWine, {lr.regParam:0.0})
modelWineB = lr.fit(trainingDataWine, {lr.regParam:100.0})


In [0]:
predictionsWineA = modelWineA.transform(testDataWine)
predictionsWineB = modelWineB.transform(testDataWine)

In [0]:
RMSE_WineA = evaluator.evaluate(predictionsWineA)
RMSE_WineB = evaluator.evaluate(predictionsWineB)

print("ModelA: Root Mean Squared Error = " + str(RMSE_WineA))
print("ModelB: Root Mean Squared Error = " + str(RMSE_WineB))

ModelA: Root Mean Squared Error = 0.6468707308612757
ModelB: Root Mean Squared Error = 0.8106450132073024
